In [ ]:
from scipy import misc



URLs = []

with open('data/urls.txt', 'r') as f:
    # read data as lines of text
    URLs = [x.splitlines()[0] for x in list(f.readlines())]
    
def pixelGrabber(url):    
    fl = cStringIO.StringIO(urllib2.urlopen(url).read())
    im = misc.imread(fl)
#     pixels = list(im)
    images.append(im)


In [ ]:
import struct
from PIL import Image
import scipy
import scipy.misc
import scipy.cluster
import cStringIO
import urllib2

NUM_CLUSTERS = 5

print 'reading image'
   
im = Image.open("images/Image1.jpg")
im = im.resize((150, 150))      # optional, to reduce time
ar = scipy.misc.fromimage(im)
shape = ar.shape
ar = ar.reshape(scipy.product(shape[:2]), shape[2])

print 'finding clusters'
codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
print 'cluster centres:\n', codes

vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences

index_max = scipy.argmax(counts)                    # find most frequent
peak = codes[index_max]
colour = ''.join(chr(c) for c in peak).encode('hex')
print 'most frequent is %s (#%s)' % (peak, colour)

In [17]:
import struct
from PIL import Image
import scipy
import scipy.misc
import scipy.cluster

NUM_CLUSTERS = 5

print 'reading image'
im = Image.open('images/Image1.jpg')
colors = Image.getpalette(im)

# shape = ar.shape
# ar = ar.reshape(scipy.product(shape[:2]), shape[2])


# print 'finding clusters'
# codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
# print 'cluster centres:\n', codes

# vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
# counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences

# index_max = scipy.argmax(counts)                    # find most frequent
# peak = codes[index_max]
# colour = ''.join(chr(c) for c in peak).encode('hex')
# print 'most frequent is %s (#%s)' % (peak, colour)

reading image


AttributeError: 'module' object has no attribute 'getpalette'

In [16]:
print colors

None


In [ ]:
from multiprocessing import Pool, TimeoutError
import time
import os
import csv

if __name__ == '__main__':
    pool = Pool(processes=4)              # start 4 worker processes
    
    URLs = []

    with open('data/urls.txt', 'r') as f:
        # read data as lines of text
        URLs = [x.splitlines()[0] for x in list(f.readlines())]
    
    with open('data/output.csv', 'wb') as csvfile:
        fieldnames = ['url', 'color1', 'color2', 'color3']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        writer.writerow(pool.map(pixelGrabber, URLs))

    # print same numbers in arbitrary order
    for i in pool.imap_unordered(f, range(10)):
        print i

    # evaluate "f(20)" asynchronously
    res = pool.apply_async(f, (20,))      # runs in *only* one process
    print res.get(timeout=1)              # prints "400"

    # evaluate "os.getpid()" asynchronously
    res = pool.apply_async(os.getpid, ()) # runs in *only* one process
    print res.get(timeout=1)              # prints the PID of that process

    # launching multiple evaluations asynchronously *may* use more processes
    multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
    print [res.get(timeout=1) for res in multiple_results]

    # make a single worker sleep for 10 secs
    res = pool.apply_async(time.sleep, (10,))
    try:
        print res.get(timeout=1)
    except TimeoutError:
        print "We lacked patience and got a multiprocessing.TimeoutError"


In [ ]:
from multiprocessing import Pool

if __name__ == '__main__':
    p = Pool(5)
    print(p.map(pixelGrabber, URLs))